Association analysis is the task of finding interesting relationships in large datasets.

The **support** of an itemset is defined as the percentage of the dataset that contains this itemset. 

 The **confidence** is defined for an association rule like {diapers}  ➞  {wine}. The confidence  for  this  rule  is  defined  as  support({diapers,  wine})/support({diapers}).

> If an itemset is infrequent, then its supersets are also infrequent.

## Finding frequent itemsets with the Apriori algorithm

In [1]:
# create a simple dataset for testing
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [2]:
# C1 is a candidate itemset of size one
# 
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item]) # add a list contain one item to list
    C1.sort()
    return list(map(frozenset, C1))
# frozenset are immutable, need to use the type as the key in a dictionary

In [3]:
# scan the dataset to see if these one itemsets meet our minimum support requirements
# dataSet, items, minimum support value
def scanD(D, Ck, minSupport):
    # create a empty dictionary
    ssCnt = {}
    # iterate every combination in dataset
    for tid in D:
        # iterate each relationship in Ck
        for can in Ck:
            # if dataset include relationship
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can]=1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData
    # return the items met the minimum support requirement
    # a dictionary with support values

In [4]:
dataSet = loadDataSet()
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [6]:
L1,suppData0 = scanD(D,C1,0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [7]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

### Putting together the full Apriori algorithm

In [8]:
# a list of frequent itemsets, and the size of the itemsets k.
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j]) # set union
    return retList

In [9]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport) # genearate a list of candidate itemsets
    L = [L1]
    k = 2
    while (len(L[k-2])>0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [10]:
L, suppData = apriori(dataSet)

In [11]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [12]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

## Mining association rules from frequent item sets 

In [13]:
# a list of frequent itemsets, a dictionary of support data for those itemsets
# and a minimum confidence threshold
def generateRules(L, supportData, minConf = 0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [14]:
def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq,'--->', conseq, 'conf:', conf)
            br1.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [15]:
# mer
def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7):
    m = len(H[0])
    if (len(freqSet)>(m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if (len(Hmp1)>1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [16]:
L, suppData = apriori(dataSet, minSupport = 0.5)
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) ---> frozenset({2}) conf: 1.0
frozenset({2}) ---> frozenset({5}) conf: 1.0
frozenset({1}) ---> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [17]:
rules=generateRules(L,suppData, minConf=0.5)

frozenset({3}) ---> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) ---> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) ---> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) ---> frozenset({2}) conf: 1.0
frozenset({2}) ---> frozenset({5}) conf: 1.0
frozenset({3}) ---> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) ---> frozenset({3}) conf: 1.0
frozenset({5}) ---> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) ---> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) ---> frozenset({3, 5}) conf: 0.6666666666666666
